Imporot knihovny ultralytics

In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.131 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.4/235.7 GB disk)


Nahrání datasetu (verze s google diskem)

In [ ]:
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

# Cesta k datasetu na Google Disku
zip_soubor = "/content/drive/MyDrive/FAVka/3.ročník/DatasetYoloFull.zip"
cilova_slozka = "Dataset_detection"

os.makedirs(cilova_slozka, exist_ok=True)

with zipfile.ZipFile(zip_soubor, 'r') as zip_ref:
    zip_ref.extractall(cilova_slozka)

print(f"✅ Soubor {zip_soubor} byl rozbalen do složky: {cilova_slozka}")


Mounted at /content/drive


Trénováání detekčního modelu YOLO11

Možnost použít následující modely:
1.   yolo11n.pt
1.   yolo11s.pt
1.   yolo11m.pt
2.   yolo11l.pt
2.   yolo11x.pt




In [ ]:
!yolo train model=yolo11s.pt data=/content/Dataset_detection/DatasetYoloFull/data.yaml epochs=10 imgsz=640

Trénování klasifikačního modelu YOLO11

Možnost použít následující modely:


1.   yolo11n-cls.pt
1.   yolo11s-cls.pt
2.   yolo11m-cls.pt
2.   yolo11l-cls.pt
2.   yolo11x-cls.pt



In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n-cls.pt")
results = model.train(data="/content/Dataset_classification/dataset_frames", epochs=30, imgsz=640)

Klasifikační predikce s grafem pro video

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO
from PIL import Image

# Parametry
video_path = '/content/video.mp4'
prag = 0.7  # <- nastav si práh, od kdy se má klasifikace zobrazit

# Výstupy modelu
results = model(video_path, stream=True)

# Video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Výstupní video
out_height = height + 300  # 300px pro graf
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, out_height))

# Data pro graf
all_probs = []
labels_dict = None
frame_index = 0

for result in results:
    ret, frame = cap.read()
    if not ret:
        break

    # Pravděpodobnosti + labely
    probs = result.probs.data.tolist()
    labels_dict = result.names
    all_probs.append(probs)

    # 🧠 Vyber nejpravděpodobnější třídu
    max_index = int(np.argmax(probs))
    max_prob = probs[max_index]
    label = labels_dict[max_index]

    # 🖋️ Pokud přesáhne práh, vykresli do rohu
    if max_prob >= prag:
        text = f"{label} ({max_prob:.0%})"
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    # 📊 Data pro graf
    arr = np.array(all_probs).T  # každá řádka = 1 třída

    # 📈 Vykreslení grafu
    fig, ax = plt.subplots(figsize=(width / 100, 3))
    for i, y in enumerate(arr):
        ax.plot(y, label=labels_dict[i])
    ax.set_ylim(0, 1)
    ax.set_xlim(0, len(all_probs))
    ax.legend(loc='upper right')
    ax.set_title(f'Snímek: {frame_index}')
    plt.tight_layout()

    # Uložení grafu jako obrázku
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    graph_img = Image.open(buf)
    graph_img = graph_img.resize((width, 300))
    graph_np = np.array(graph_img)
    graph_np = cv2.cvtColor(graph_np, cv2.COLOR_RGB2BGR)

    # Spojení videa a grafu
    combined = np.vstack((frame, graph_np))
    out.write(combined)
    frame_index += 1

cap.release()
out.release()

print("✅ Hotovo! Výstupní video je uložené jako 'output.mp4'")


Detekční predikce pro video

Možnost nastavení jak moc se mohou detekce překrývat (iou) a s jako přesností se budou predikce objevovat (conf)

In [ ]:
!yolo predict model=/content/runs/detect/train/weights/best.pt source='/content/video4.mp4' conf=0.60 iou=0.3 save=True